In [15]:
import requests
import json
import pandas as pd

In [30]:
def model_predict(data_original):
    
    data = data_original.copy()
    if isinstance(data, pd.DataFrame):
        data['GPA'] = data['GPA'].apply(lambda x: f"{x:.2f}")
        data = data.to_dict(orient='records')
    # Serialize the input data to JSON
    dataset = json.dumps(data)
    
    # Define the headers for JSON content type
    headers = {'Content-Type': 'application/json'}
    
    # Call the first API - resume scorer
    resume_url = 'https://jennjwang.pythonanywhere.com'
    resume_response = requests.post(resume_url, data=dataset, headers=headers)
    
    if not resume_response.ok:
        print("Error:", resume_response.status_code)
        return None
    
    try:
        resume_response_data = json.loads(resume_response.text)
        resume_predictions = json.loads(resume_response_data['prediction'])
        resume_score_map = {item['applicant_id']: item['score'] for item in resume_predictions}
    except json.JSONDecodeError as e:
        print("Error decoding JSON:", e)
        return None
    

    # Update the input data with the resume score
    for applicant in data:
        applicant_id = str(applicant['Applicant ID'])
        applicant['Resume score'] = resume_score_map.get(applicant_id, 0)
    
    # Serialize the updated data for the next API call
    updated_dataset = json.dumps(data)

    return updated_dataset
    
    # Call the second API - candidate scorer
    candidate_url = 'https://heonlee.pythonanywhere.com'
    candidate_response = requests.post(candidate_url, data=updated_dataset, headers=headers)
    
    if not candidate_response.ok:
        print("Error:", candidate_response.status_code)
        return None
    
    try:
        candidate_response_data = json.loads(candidate_response.text)
        final_predictions = json.loads(candidate_response_data['prediction'])
        final_score_map = {item['applicant_id']: item['prediction'] for item in final_predictions}
    except json.JSONDecodeError as e:
        print("Error decoding JSON for the second API:", e)
        return None
    
    
    for applicant in data:
        applicant_id = str(applicant['Applicant ID'])
        applicant['Interview prediction'] = final_score_map.get(applicant_id, 0)
        
    results = pd.DataFrame(data)
    results['GPA'] = results['GPA'].astype(float)
    results['Resume score'] = results['Resume score'].astype(float)
    results['Interview prediction'] = results['Interview prediction'].astype(int)
    
    return results

In [36]:
df = pd.read_csv('../data/candidate_data.csv')
# df_prediction = model_predict(df)
# df_prediction

In [37]:
df

,Applicant ID,School Name,GPA,Degree,Location,Gender,Veteran status,Work authorization,Disability,Ethnicity,Role 1,Start 1,End 1,Role 2,Start 2,End 2,Role 3,Start 3,End 3
0,1,Rhode Island School of Design,3.1,Bachelors,Miami,M,1.0,0,0.0,2,ML Engineer,7/16,7/22,Junior SWE,11/19,11/22,Senior SWE,12/21,12/22
1,2,Bentley University,1.9,Phd,New York City,NaN,0.0,0,1.0,0,Bus Driver,11/13,11/18,ML Engineer,11/14,12/14,NaN,NaN,NaN
2,3,SUNY New Paltz,1.7,Masters,Washington D.C.,M,1.0,1,1.0,1,Data scientist,4/22,6/22,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Brown University,2.8,Bachelors,Los Angeles,F,1.0,0,1.0,2,Senior SWE,6/22,11/22,Lawyer,1/12,4/12,Lawyer,8/13,8/14
4,5,SUNY Binghamton University,2.8,Masters,Los Angeles,NaN,1.0,1,NaN,4,Lawyer,6/11,NaN,Junior SWE,6/10,7/10,Junior SWE,7/23,10/23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,3996,Rhode Island School of Design,2.3,Masters,Detroit,F,1.0,0,NaN,0,ML Engineer,3/15,3/18,Lawyer,5/21,5/22,NaN,NaN,NaN
3996,3997,SUNY Binghamton University,1.6,Phd,Providence,NaN,0.0,0,NaN,0,Lawyer,10/17,10/18,Data scientist,5/21,5/22,Junior SWE,5/16,5/22
3997,3998,SUNY New Paltz,1.2,Bachelors,New York City,F,0.0,1,NaN,4,Chef,8/10,8/17,NaN,NaN,NaN,NaN,NaN,NaN
3998,3999,Brown University,1.8,Bachelors,Washington D.C.,NaN,0.0,1,1.0,4,Bus Driver,1/16,1/20,Chef,9/11,9/13,Senior SWE,9/14,9/22


In [35]:
# Parse the JSON data
data = json.loads(df_prediction)

# Convert JSON to DataFrame
df = pd.DataFrame(data)

df

,Applicant ID,School Name,GPA,Degree,Location,Gender,Veteran status,Work authorization,Disability,Ethnicity,Role 1,Start 1,End 1,Role 2,Start 2,End 2,Role 3,Start 3,End 3,Resume score
0,1,Rhode Island School of Design,3.10,Bachelors,Miami,M,1.0,0,0.0,2,ML Engineer,7/16,7/22,Junior SWE,11/19,11/22,Senior SWE,12/21,12/22,5.56
1,2,Bentley University,1.90,Phd,New York City,NaN,0.0,0,1.0,0,Bus Driver,11/13,11/18,ML Engineer,11/14,12/14,NaN,NaN,NaN,2.99
2,3,SUNY New Paltz,1.70,Masters,Washington D.C.,M,1.0,1,1.0,1,Data scientist,4/22,6/22,NaN,NaN,NaN,NaN,NaN,NaN,4.88
3,4,Brown University,2.80,Bachelors,Los Angeles,F,1.0,0,1.0,2,Senior SWE,6/22,11/22,Lawyer,1/12,4/12,Lawyer,8/13,8/14,4.34
4,5,SUNY Binghamton University,2.80,Masters,Los Angeles,NaN,1.0,1,NaN,4,Lawyer,6/11,NaN,Junior SWE,6/10,7/10,Junior SWE,7/23,10/23,1.84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,3996,Rhode Island School of Design,2.30,Masters,Detroit,F,1.0,0,NaN,0,ML Engineer,3/15,3/18,Lawyer,5/21,5/22,NaN,NaN,NaN,8.65
3996,3997,SUNY Binghamton University,1.60,Phd,Providence,NaN,0.0,0,NaN,0,Lawyer,10/17,10/18,Data scientist,5/21,5/22,Junior SWE,5/16,5/22,2.74
3997,3998,SUNY New Paltz,1.20,Bachelors,New York City,F,0.0,1,NaN,4,Chef,8/10,8/17,NaN,NaN,NaN,NaN,NaN,NaN,4.43
3998,3999,Brown University,1.80,Bachelors,Washington D.C.,NaN,0.0,1,1.0,4,Bus Driver,1/16,1/20,Chef,9/11,9/13,Senior SWE,9/14,9/22,5.4
